<a href="https://colab.research.google.com/github/ficle-fr/pix2pix_change_style/blob/colabs/colabs/train_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/common.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/descriminator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/generator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/img_generator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/train.py

#Currently from the colab branch
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/colabs/colabs/read_write_db.py


In [ ]:
!sudo apt install libcairo2-dev pkg-config python3-dev
!pip install pycairo

In [ ]:
import tensorflow as tf
import sys
#sys.path.append('/content/pix2pix_change_style')
from train import fit, train_step

from generator import Generator, generator_loss
from descriminator import Discriminator, discriminator_loss

from img_generator import img_pair_gen1
from read_write_db import write, read


In [ ]:
write("temp_records.tfrecords", 10)

In [ ]:
dataset = read("./temp_records.tfrecords")
print(dataset)
for _, data_batch in zip(range(2), dataset):
    print(data_batch)


In [ ]:
generator = Generator([256, 256, 3], 3)
discriminator = Discriminator([256, 256, 3])

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
def checkpoint_callback():
  pass

BATCH_SIZE = 5
dataset = tf.data.Dataset.from_generator(
    lambda: img_pair_gen1(256, 256), output_signature = (
        tf.TensorSpec(shape = (256, 256, 3), dtype = tf.float32),
        tf.TensorSpec(shape = (256, 256, 3), dtype = tf.float32)))
dataset = dataset.batch(BATCH_SIZE)

#fit(generator, discriminator, generator_optimizer, discriminator_optimizer,
#    dataset, dataset, 40, 50, checkpoint_callback)
fit(generator, discriminator, generator_optimizer, discriminator_optimizer,
    dataset, dataset, 2, 10, checkpoint_callback)
